In [ ]:
# Đọc dữ liệu từ tập tin CSV
import pandas as pd
# Đường dẫn đến tập tin CSV
file_path = 'C:\\Users\\ACER\\Desktop\\BD_DQ\\data_dotquy.csv'#Hien
#file_path = 'C:\\Users\\Admin\\Desktop\\BD_DQ\\data_dotquy.csv'
# Đọc dữ liệu từ CSV vào DataFrame
df = pd.read_csv(file_path)
# Hiển thị DataFrame trong VS Code
print(df)

LÀM SẠCH và CHUẨN HÓA DL
- 1 giới tính: loại other
- 2 tuổi: Loại số thực ( vd: 0.1 , 1.56) rồi dùng kỹ thuật Age Grouping
- 3 Đã kết hôn: chuyển về 0 và một
- 4 kiểu làm việc(work_type)
    + kỹ thuật Hashing Encoding
    + kỹ thuật One-Hot Encoding
- 5 vùng sống: chuyển về 0 và một
- 6 BMI  
    + Loại bỏ các hàng hoặc cột có N/A rồi dùng kỹ thuật Binary Encoding
    + dùng kỹ thuật Simple Imputation rồi dùng kỹ thuật tiếp Min-Max Scaling
- 7 smoking_status
  + Loại bỏ các hàng hoặc cột có Unknown rồi dùng kỹ thuật tiếp Min-Max Scaling
  + kỹ thuật Frequency Encoding
- 8 avg_glucose_level (mg/dL)
    DÙNG KỸ THUẬT: Min-Max Scaling
  ******ĐIỀU KIỆN LÀ******
    Người bình thường:
     < 140
    Người Bị tiểu đường:
      >=140

In [ ]:
#Hien
#Kiem tra trung lap
duplicated_rows = df[df.duplicated(subset='id')]
print("So luong du lieu trung lap la:" , len(duplicated_rows))

df = df.dropna()
df = df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x )

In [ ]:
#Hien
# Xóa cột mã định danh
df  = df.drop(columns=['id'])
df.info()

In [ ]:
#Chau
#Loại bỏ giới tính mang giá trị Other
mask = df['gender'].isin(['Male', 'Female'])
df = df[mask]
print(df)

In [ ]:
#Chau
# gender: female 1 male 0
df.loc[:, 'gender'] = df['gender'].replace({'Female': 1, 'Male': 0})
print(df)

In [ ]:
#Chau
# Residence_type: urban 1 rural 0
df.loc[:, 'Residence_type'] = df['Residence_type'].replace({'Urban': 1, 'Rural': 0})

#Chi
# Kiểm tra xem tuổi có phải là số nguyên hay không
df['Is_Whole_Number'] = df['age'].apply(lambda x: x.is_integer())

# Lọc các hàng mà tuổi không phải là số thực (chỉ giữ lại số nguyên)
df = df[df['Is_Whole_Number']]

# Xóa cột tạm thời 'Is_Whole_Number'
df = df.drop(columns=['Is_Whole_Number'])

df

In [ ]:
#Chi
# Định nghĩa các khoảng tuổi và các nhãn cho các nhóm tuổi
bins = [0, 18, 35, 60, 100]
labels = ['0', '1', '2', '3']

# Chia cột "Age" thành các nhóm tuổi
df['Age Group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

df

In [ ]:
#Hien
# khong tieu duong
def glucose_normal(value):
    if value < 140:
        return value
    else:
        return 0
normal_data = [glucose_normal(x) for x in df['avg_glucose_level']]
#print(normal_data)

# Calculate min and max values
X_min = min(normal_data)
X_max = max(normal_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in normal_data]
scaled_data
df['glucose_normal'] = scaled_data


In [ ]:
#Hien
# tieu duong
def glucose_classify(value):
    if value >= 140:
        return value
    else:
        return 0

classified_data = [glucose_classify(x) for x in df['avg_glucose_level']]
#classified_data

# Calculate min and max values
X_min = min(classified_data)
X_max = max(classified_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in classified_data]
scaled_data
df['glucose_classify'] = scaled_data


In [ ]:
#Hien
# chuẩn hóa bằng Min-Max Scaling
glucose_values = df['avg_glucose_level']

# Calculate min and max values
X_min = min(df['avg_glucose_level'])
X_max = max(df['avg_glucose_level'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['avg_glucose_level']]

# Print the original and scaled data
#print(scaled_data)
#df['glucose_normalized_data'] = scaled_data

df['avg_glucose_level'] = scaled_data
df


In [ ]:

# LOAI GIA TRI TUOI LÀ SO THỤC KHONG NGUYEN
df = df[~df['age'].apply(lambda x: isinstance(x, float) and not x.is_integer())]
df

In [ ]:
#Chau
#chuẩn hóa dữ liệu field work_type theo kỹ thuật label encoding
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df.loc[:, 'work_type'] = le.fit_transform(df['work_type'])
print(df)

#Chi
# Chuyển đổi các nhóm tuổi thành các giá trị số
df['Age Group'] = df['Age Group'].cat.codes.astype(float)
df

In [ ]:
#Giang 

df['Residence_type']=df['Residence_type'].replace({'Urban':1,'Rural':0})
mask = df['gender'] != 'Other'
df_filtered = df[mask]
print(df_filtered) 

In [ ]:
#Giang
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['work_type'] = label_encoder.fit_transform(df['work_type'])

# sử dụng chuẩn hóa bằng Min-Max Scaling chuan hoa df['work_type'] dang (0 1 2 3 4)
# Calculate min and max values
X_min = min(df['work_type'])
X_max = max(df['work_type'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['work_type']]
scaled_data
# Tạo ra trường dữ liệu mới work_type_2
df['work_type_2'] =scaled_data
df

In [ ]:
#Hoang
# Dữ liệu ban đầu
data = df['bmi']

# Min-Max Scaling
data_min = data.min()
data_max = data.max()

scaled_data = (data - data_min) / (data_max - data_min)

# Hiển thị dữ liệu đã chuẩn hóa
print(scaled_data)

In [ ]:
#Hoang
df['bmi']=scaled_data
df

In [ ]:
#Hoang
# Dữ liệu ban đầu
data = df['bmi']

# Chuyển đổi dữ liệu thành DataFrame
dff = pd.DataFrame(data, columns=['bmi'])
dff

In [ ]:
#Hoang
import numpy as np
conditions = [
    (dff['bmi'] < 18.5),
    (dff['bmi'] >= 18.5) & (dff['bmi'] <= 25),
    (dff['bmi'] > 25)
]

# Các giá trị tương ứng cho mỗi điều kiện
df['underweight'] = np.where(dff['bmi'] < 18.5, 1, 0)
df['normal'] = np.where((dff['bmi'] >= 18.5) & (dff['bmi'] <= 25), 1, 0)
df['overweight'] = np.where(dff['bmi'] > 25, 1, 0)

# Hiển thị DataFrame với dữ liệu đã chuẩn hóa
df

In [ ]:
#Binh
df.dropna(how='all', inplace=True)
df.head()

In [ ]:
#Binh
df = df[df['smoking_status'] != 'Unknown']
df

In [ ]:
#Binh
#sử dụng labelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['smoking_status'] = le.fit_transform(df['smoking_status'])
#fit_tranform tự động gán giá trị cho smoking_status
df